In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from opacus import PrivacyEngine
from opacus.accountants.analysis import rdp as privacy_analysis
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim

In [31]:
# Set random seeds for reproducibility
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility across all libraries."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed set to {seed}")

RANDOM_SEED = 42
set_random_seeds(RANDOM_SEED)

Random seed set to 42


# Import Dataset

In [32]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

# Data Cleaning

# Data Preprocessing

In [33]:
y = y.iloc[:, 0]
# Strip spaces and remove periods
y = y.str.strip().str.replace('.', '', regex=False)

# Map both <=50K and <50K to 0; >=50K and >50K to 1
y = y.replace({
    '<=50K': 0,
    '<50K': 0,
    '>=50K': 1,
    '>50K': 1
})

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(exclude=['object']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

## Fairness Metrics Function

In [34]:
def fairness_metrics(X_test, y_true, y_pred, feature):
    """
    Calculate fairness metrics for a specific sensitive feature.
    
    Returns demographic parity, equal opportunity (TPR), and equalized odds (FPR)
    for each group in the feature.
    """
    df_eval = X_test.copy()
    df_eval['y_true'] = y_true.values if hasattr(y_true, 'values') else y_true
    df_eval['y_pred'] = y_pred
    
    groups = df_eval[feature].unique()
    metrics = []

    for g in groups:
        mask = df_eval[feature] == g
        y_t = df_eval.loc[mask, 'y_true']
        y_p = df_eval.loc[mask, 'y_pred']

        # Positive prediction rate (Demographic Parity)
        pos_rate = np.mean(y_p)

        # True Positive Rate (Equal Opportunity)
        tp = np.sum((y_p == 1) & (y_t == 1))
        fn = np.sum((y_p == 0) & (y_t == 1))
        tpr = tp / (tp + fn + 1e-10)

        # False Positive Rate (Equalized Odds)
        fp = np.sum((y_p == 1) & (y_t == 0))
        tn = np.sum((y_p == 0) & (y_t == 0))
        fpr = fp / (fp + tn + 1e-10)

        # Accuracy
        acc = np.mean(y_p == y_t)

        metrics.append({
            'group': g,
            'positive_rate': pos_rate,
            'TPR': tpr,
            'FPR': fpr,
            'accuracy': acc
        })

    df_metrics = pd.DataFrame(metrics)
    print(f"\n=== Fairness by {feature} ===")
    print(df_metrics.to_string(index=False))

    # Calculate gaps between groups
    max_min_gap = df_metrics[['positive_rate', 'TPR', 'FPR', 'accuracy']].max() - \
                  df_metrics[['positive_rate', 'TPR', 'FPR', 'accuracy']].min()
    print("\nGaps between groups (max - min):")
    print(max_min_gap.to_string())

    return df_metrics


def evaluate_model_full(name, model, X_model_input, y_true, X_sensitive, 
                        fairness_metrics_func=fairness_metrics):
    """
    Evaluate a model with standard metrics and fairness metrics.
    
    Works with both sklearn models and PyTorch models.
    """
    # Detect if PyTorch model
    is_torch_model = isinstance(model, torch.nn.Module)

    if is_torch_model:
        # Ensure tensor input
        if not torch.is_tensor(X_model_input):
            X_model_input = torch.tensor(X_model_input, dtype=torch.float32)
        model.eval()
        with torch.no_grad():
            outputs = model(X_model_input)
            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            y_pred = np.argmax(outputs.cpu().numpy(), axis=1)
    else:
        # sklearn model
        y_pred = model.predict(X_model_input)
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X_model_input)[:, 1]
        else:
            probs = y_pred

    # Standard metrics
    acc = accuracy_score(y_true, y_pred)
    try:
        auc = roc_auc_score(y_true, probs)
    except ValueError:
        auc = float('nan')

    print(f"\n{'='*70}")
    print(f"EVALUATION: {name}")
    print(f"{'='*70}")
    print(f"Overall Accuracy: {acc:.4f}")
    print(f"AUC-ROC Score: {auc:.4f}")

    # Fairness metrics
    sex_metrics = race_metrics = None
    if fairness_metrics_func and X_sensitive is not None:
        if 'sex' in X_sensitive.columns:
            sex_metrics = fairness_metrics_func(X_sensitive, y_true, y_pred, 'sex')
        if 'race' in X_sensitive.columns:
            race_metrics = fairness_metrics_func(X_sensitive, y_true, y_pred, 'race')

    return {
        "name": name,
        "accuracy": acc,
        "auc": auc,
        "y_pred": y_pred,
        "sex_metrics": sex_metrics,
        "race_metrics": race_metrics
    }

## Pytorch Model Definition

In [35]:
class LogisticRegressionModel(nn.Module):
    """Simple logistic regression model for binary classification."""
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 2)

    def forward(self, x):
        return self.linear(x)

# BASELINE MODEL (NO DIFFERENTIAL PRIVACY)

In [36]:
def train_baseline_model(X_train, y_train, X_test, y_test, 
                         max_iter=1000, random_state=RANDOM_SEED):
    """
    Train a baseline logistic regression model without any privacy protection.
    
    Args:
        X_train, y_train: Training data (pandas DataFrames/Series)
        X_test, y_test: Test data
        max_iter: Maximum iterations for sklearn LogisticRegression
        random_state: Random seed
        
    Returns:
        model: Trained sklearn pipeline
        results: Evaluation results dictionary
    """
    print(f"\n{'='*70}")
    print("TRAINING BASELINE MODEL (No Differential Privacy)")
    print(f"{'='*70}")
    
    # Create preprocessing + model pipeline
    clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=max_iter, random_state=random_state))
    ])
    
    # Train
    clf.fit(X_train, y_train)
    print("✓ Training completed")
    
    # Evaluate
    results = evaluate_model_full(
        name="Baseline (No DP)",
        model=clf,
        X_model_input=X_test,
        y_true=y_test,
        X_sensitive=X_test
    )
    
    return clf, results


# STANDARD DP-SGD MODEL (OPACUS)

In [37]:
def train_standard_dpsgd(X_train_tensor, y_train_tensor, X_test_tensor, y_test,
                         X_test_df, noise_multiplier=1.0, max_grad_norm=1.0,
                         lr=0.01, epochs=10, batch_size=64, delta=1e-5):
    """
    Train a logistic regression model with standard DP-SGD using Opacus.
    All gradients are protected with differential privacy.
    
    Args:
        X_train_tensor: Preprocessed training features (torch tensor)
        y_train_tensor: Training labels (torch tensor)
        X_test_tensor: Preprocessed test features (torch tensor)
        y_test: Test labels (pandas Series or numpy array)
        X_test_df: Original test dataframe for fairness evaluation
        noise_multiplier: Controls noise magnitude (higher = more privacy)
        max_grad_norm: Gradient clipping threshold (lower = more privacy)
        lr: Learning rate
        epochs: Number of training epochs
        batch_size: Batch size
        delta: Privacy parameter (typically 1e-5)
        
    Returns:
        model: Trained PyTorch model
        epsilon: Privacy budget (ε)
        results: Evaluation results dictionary
    """
    print(f"\n{'='*70}")
    print("TRAINING STANDARD DP-SGD MODEL")
    print(f"{'='*70}")
    print(f"Hyperparameters:")
    print(f"  - Learning rate: {lr}")
    print(f"  - Epochs: {epochs}")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Noise multiplier: {noise_multiplier}")
    print(f"  - Max grad norm: {max_grad_norm}")
    print(f"  - Delta (δ): {delta}")
    
    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model
    model = LogisticRegressionModel(X_train_tensor.shape[1])
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # Attach Privacy Engine
    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
    )
    
    # Training loop
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    print("Training completed")
    
    # Calculate privacy budget
    epsilon = privacy_engine.get_epsilon(delta=delta)
    print(f"\n{'='*70}")
    print("PRIVACY BUDGET")
    print(f"{'='*70}")
    print(f"Epsilon (ε): {epsilon:.4f}")
    print(f"Delta (δ): {delta}")
    print(f"\nInterpretation:")
    print(f"  - ALL features are protected with ε={epsilon:.4f} privacy")
    print(f"  - Lower ε = stronger privacy (typical target: ε ≤ 1.0)")
    
    # Evaluate
    results = evaluate_model_full(
        name=f"Standard DP-SGD (ε={epsilon:.3f})",
        model=model,
        X_model_input=X_test_tensor,
        y_true=y_test,
        X_sensitive=X_test_df
    )
    
    results['epsilon'] = epsilon
    results['delta'] = delta
    
    return model, epsilon, results

# SELECTIVE DP-SGD MODEL (MANUAL IMPLEMENTATION)

In [39]:
def train_selective_dpsgd(X_train_tensor, y_train_tensor, X_test_tensor, y_test,
                          X_test_df, sensitive_indices, noise_multiplier=1.0,
                          max_grad_norm=1.0, lr=0.01, epochs=10, batch_size=64,
                          delta=1e-5):
    """
    Train a logistic regression model with Selective DP-SGD.
    Only gradients corresponding to sensitive features are protected with DP noise.
    
    Args:
        X_train_tensor: Preprocessed training features (torch tensor)
        y_train_tensor: Training labels (torch tensor)
        X_test_tensor: Preprocessed test features (torch tensor)
        y_test: Test labels
        X_test_df: Original test dataframe for fairness evaluation
        sensitive_indices: List of feature indices to protect (e.g., sex, race)
        noise_multiplier: Controls noise magnitude (higher = more privacy)
        max_grad_norm: Gradient clipping threshold (lower = more privacy)
        lr: Learning rate
        epochs: Number of training epochs
        batch_size: Batch size
        delta: Privacy parameter
        
    Returns:
        model: Trained PyTorch model
        epsilon: Privacy budget for sensitive features
        results: Evaluation results dictionary
    """
    print(f"\n{'='*70}")
    print("TRAINING SELECTIVE DP-SGD MODEL")
    print(f"{'='*70}")
    print(f"Hyperparameters:")
    print(f"  - Learning rate: {lr}")
    print(f"  - Epochs: {epochs}")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Noise multiplier: {noise_multiplier}")
    print(f"  - Max grad norm: {max_grad_norm}")
    print(f"  - Delta (δ): {delta}")
    print(f"  - Sensitive feature indices: {sensitive_indices}")
    
    # Initialize model
    model = LogisticRegressionModel(X_train_tensor.shape[1])
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # Training loop with selective DP
    n_samples = len(X_train_tensor)
    n_batches = (n_samples + batch_size - 1) // batch_size
    
    model.train()
    for epoch in range(epochs):
        # Shuffle data
        indices = torch.randperm(n_samples)
        X_shuffled = X_train_tensor[indices]
        y_shuffled = y_train_tensor[indices]
        
        epoch_loss = 0
        for batch_idx in range(n_batches):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, n_samples)
            
            X_batch = X_shuffled[start_idx:end_idx]
            y_batch = y_shuffled[start_idx:end_idx]
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            
            # Apply selective DP to gradients
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if "linear.weight" in name and param.grad is not None:
                        grad = param.grad
                        
                        # Create mask for sensitive features
                        mask = torch.zeros_like(grad)
                        mask[:, sensitive_indices] = 1.0
                        
                        # Clip sensitive feature gradients
                        sensitive_grad = grad * mask
                        grad_norm = sensitive_grad.norm(2)
                        if grad_norm > max_grad_norm:
                            # Normalize only sensitive gradients
                            clipped_sensitive = sensitive_grad * (max_grad_norm / grad_norm)
                            # Combine: non-sensitive (unchanged) + clipped sensitive
                            grad.data = grad * (1.0 - mask) + clipped_sensitive
                        
                        # Add Gaussian noise ONLY to sensitive features
                        noise = torch.normal(
                            mean=0.0,
                            std=noise_multiplier * max_grad_norm,
                            size=grad.shape,
                            device=grad.device
                        )
                        param.grad.add_(noise * mask)
            
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / n_batches
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    print("Training completed")
    
    # Calculate privacy budget for sensitive features
    steps = epochs * n_batches
    q = batch_size / n_samples  # Sampling rate
    
    # Use RDP accountant to compute APPROXIMATE epsilon
    orders = [1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64))
    rdp = privacy_analysis.compute_rdp(
        q=q,
        noise_multiplier=noise_multiplier,
        steps=steps,
        orders=orders
    )
    epsilon = privacy_analysis.get_privacy_spent(orders=orders, rdp=rdp, delta=delta)[0]
    
    print(f"\n{'='*70}")
    print("PRIVACY BUDGET (SELECTIVE)")
    print(f"{'='*70}")
    print(f"Epsilon (ε): {epsilon:.4f}")
    print(f"Delta (δ): {delta}")
    print(f"\nInterpretation:")
    print(f"  - Sensitive features (indices {sensitive_indices}) have ε={epsilon:.4f} privacy")
    print(f"  - Non-sensitive features have NO formal privacy guarantee")
    print(f"  - Lower ε = stronger privacy (typical target: ε ≤ 1.0)")
    
    # Evaluate
    results = evaluate_model_full(
        name=f"Selective DP-SGD (ε={epsilon:.3f})",
        model=model,
        X_model_input=X_test_tensor,
        y_true=y_test,
        X_sensitive=X_test_df
    )
    
    results['epsilon'] = epsilon
    results['delta'] = delta
    
    return model, epsilon, results

# MAIN EXECUTION

In [40]:
if __name__ == "__main__":
    print("\n" + "="*70)
    print("PREPROCESSING DATA FOR PYTORCH MODELS")
    print("="*70)
    
    # Preprocess features
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # Convert to torch tensors
    def to_tensor(x):
        if hasattr(x, "toarray"):
            return torch.tensor(x.toarray(), dtype=torch.float32)
        return torch.tensor(x, dtype=torch.float32)
    
    X_train_tensor = to_tensor(X_train_processed)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = to_tensor(X_test_processed)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
    
    print(f"Train tensor shape: {X_train_tensor.shape}")
    print(f"Test tensor shape: {X_test_tensor.shape}")
    
    # Identify sensitive feature indices
    print("\n" + "="*70)
    print("IDENTIFYING SENSITIVE FEATURES")
    print("="*70)
    
    encoded_feature_names = preprocessor.get_feature_names_out()
    sensitive_indices = []
    for idx, name in enumerate(encoded_feature_names):
        if name.startswith('cat__sex_') or name.startswith('cat__race_'):
            sensitive_indices.append(idx)
    
    print(f"Total features after encoding: {len(encoded_feature_names)}")
    print(f"Sensitive feature indices: {sensitive_indices}")
    print(f"Sensitive feature names:")
    for idx in sensitive_indices:
        print(f"  [{idx}] {encoded_feature_names[idx]}")


PREPROCESSING DATA FOR PYTORCH MODELS
✓ Train tensor shape: torch.Size([39073, 111])
✓ Test tensor shape: torch.Size([9769, 111])

IDENTIFYING SENSITIVE FEATURES
Total features after encoding: 111
Sensitive feature indices: [61, 62, 63, 64, 65, 66, 67]
Sensitive feature names:
  [61] cat__race_Amer-Indian-Eskimo
  [62] cat__race_Asian-Pac-Islander
  [63] cat__race_Black
  [64] cat__race_Other
  [65] cat__race_White
  [66] cat__sex_Female
  [67] cat__sex_Male


In [41]:
#1. Baseline Model
baseline_model, baseline_results = train_baseline_model(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    max_iter=1000
)


TRAINING BASELINE MODEL (No Differential Privacy)
✓ Training completed

EVALUATION: Baseline (No DP)
Overall Accuracy: 0.8517
AUC-ROC Score: 0.9058

=== Fairness by sex ===
 group  positive_rate      TPR      FPR  accuracy
  Male       0.256173 0.617453 0.098248  0.815278
Female       0.075707 0.495913 0.022930  0.923381

Gaps between groups (max - min):
positive_rate    0.180466
TPR              0.121540
FPR              0.075318
accuracy         0.108103

=== Fairness by race ===
             group  positive_rate      TPR      FPR  accuracy
             White       0.209391 0.606890 0.074169  0.844873
             Other       0.072464 0.363636 0.017241  0.884058
             Black       0.072034 0.421488 0.020656  0.907839
Asian-Pac-Islander       0.266447 0.679487 0.123894  0.825658
Amer-Indian-Eskimo       0.067308 0.555556 0.021053  0.942308

Gaps between groups (max - min):
positive_rate    0.199140
TPR              0.315851
FPR              0.106652
accuracy         0.116650


In [42]:
# 2. Standard DP-SGD Model
dpsgd_model, dpsgd_epsilon, dpsgd_results = train_standard_dpsgd(
        X_train_tensor=X_train_tensor,
        y_train_tensor=y_train_tensor,
        X_test_tensor=X_test_tensor,
        y_test=y_test,
        X_test_df=X_test,
        noise_multiplier=1.0,
        max_grad_norm=1.0,
        lr=0.01,
        epochs=10,
        batch_size=64,
        delta=1e-5
    )


TRAINING STANDARD DP-SGD MODEL
Hyperparameters:
  - Learning rate: 0.01
  - Epochs: 10
  - Batch size: 64
  - Noise multiplier: 1.0
  - Max grad norm: 1.0
  - Delta (δ): 1e-05


/Users/enricoalmadani/anaconda3/lib/python3.10/site-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


Epoch 1/10 - Loss: 0.5353
Epoch 2/10 - Loss: 0.4868
Epoch 3/10 - Loss: 0.4368
Epoch 4/10 - Loss: 0.4037
Epoch 5/10 - Loss: 0.3907
Epoch 6/10 - Loss: 0.3885
Epoch 7/10 - Loss: 0.3817
Epoch 8/10 - Loss: 0.3775
Epoch 9/10 - Loss: 0.3798
Epoch 10/10 - Loss: 0.3767
✓ Training completed

PRIVACY BUDGET
Epsilon (ε): 0.6400
Delta (δ): 1e-05

Interpretation:
  - ALL features are protected with ε=0.6400 privacy
  - Lower ε = stronger privacy (typical target: ε ≤ 1.0)

EVALUATION: Standard DP-SGD (ε=0.640)
Overall Accuracy: 0.8338
AUC-ROC Score: 0.8855

=== Fairness by sex ===
 group  positive_rate      TPR      FPR  accuracy
  Male       0.225309 0.535261 0.089820  0.796142
Female       0.033445 0.237057 0.007871  0.907875

Gaps between groups (max - min):
positive_rate    0.191864
TPR              0.298204
FPR              0.081949
accuracy         0.111733

=== Fairness by race ===
             group  positive_rate      TPR      FPR  accuracy
             White       0.167106 0.486550 0.058436

In [43]:
# 3. Selective DP-SGD Model
selective_model, selective_epsilon, selective_results = train_selective_dpsgd(
        X_train_tensor=X_train_tensor,
        y_train_tensor=y_train_tensor,
        X_test_tensor=X_test_tensor,
        y_test=y_test,
        X_test_df=X_test,
        sensitive_indices=sensitive_indices,
        noise_multiplier=1.0,
        max_grad_norm=1.0,
        lr=0.01,
        epochs=10,
        batch_size=64,
        delta=1e-5
    )


TRAINING SELECTIVE DP-SGD MODEL
Hyperparameters:
  - Learning rate: 0.01
  - Epochs: 10
  - Batch size: 64
  - Noise multiplier: 1.0
  - Max grad norm: 1.0
  - Delta (δ): 1e-05
  - Sensitive feature indices: [61, 62, 63, 64, 65, 66, 67]
Epoch 1/10 - Loss: 0.4232
Epoch 2/10 - Loss: 0.3587
Epoch 3/10 - Loss: 0.3476
Epoch 4/10 - Loss: 0.3418
Epoch 5/10 - Loss: 0.3372
Epoch 6/10 - Loss: 0.3332
Epoch 7/10 - Loss: 0.3328
Epoch 8/10 - Loss: 0.3310
Epoch 9/10 - Loss: 0.3287
Epoch 10/10 - Loss: 0.3294
✓ Training completed

PRIVACY BUDGET (SELECTIVE)
Epsilon (ε): 0.9219
Delta (δ): 1e-05

Interpretation:
  - Sensitive features (indices [61, 62, 63, 64, 65, 66, 67]) have ε=0.9219 privacy
  - Non-sensitive features have NO formal privacy guarantee
  - Lower ε = stronger privacy (typical target: ε ≤ 1.0)

EVALUATION: Selective DP-SGD (ε=0.922)
Overall Accuracy: 0.8450
AUC-ROC Score: 0.8996

=== Fairness by sex ===
 group  positive_rate      TPR      FPR  accuracy
  Male       0.245525 0.589041 0.09